In [1]:
import gevent

In [2]:
def foo():
    print('Running in foo')
    gevent.sleep(0)
    print('Explicit context switch to foo again')
    
def bar():
    print('Explicit context to bar')
    gevent.sleep(0)
    print('Implicit context switch back to bar')
    
gevent.joinall([
    gevent.spawn(foo),
    gevent.spawn(bar),
])

Running in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar


[<Greenlet at 0x7f27b03cf158: _run>, <Greenlet at 0x7f27b03cf268: _run>]

In [3]:
import time
import gevent
from gevent import select

In [4]:
start = time.time()
tic = lambda: 'at %1.1f seconds' % (time.time() - start)

def gr1():
    # Busy waits for a second, but we don't want to stick around...
    print('Started Polling: %s' % tic())
    select.select([], [], [], 2)
    print('Ended Polling: %s' % tic())

def gr2():
    # Busy waits for a second, but we don't want to stick around...
    print('Started Polling: %s' % tic())
    select.select([], [], [], 2)
    print('Ended Polling: %s' % tic())

def gr3():
    print("Hey lets do some stuff while the greenlets poll, %s" % tic())
    gevent.sleep(1)

gevent.joinall([
    gevent.spawn(gr1),
    gevent.spawn(gr2),
    gevent.spawn(gr3),
])

Started Polling: at 0.0 seconds
Started Polling: at 0.0 seconds
Hey lets do some stuff while the greenlets poll, at 0.0 seconds
Ended Polling: at 2.0 seconds
Ended Polling: at 2.0 seconds


[<Greenlet at 0x7f27b03cf598: _run>,
 <Greenlet at 0x7f27b03cf048: _run>,
 <Greenlet at 0x7f27b03cf488: _run>]

In [5]:
import threading

In [6]:
x = 0

In [8]:
def foo():
    global x
    for i in range(10000000):
        x += 1

def bar():
    global x
    for i in range(10000000):
        x -= 1
    
t1 = threading.Thread(target=foo)
t2 = threading.Thread(target=bar)
t1.start()
t2.start()
t1.join()
t2.join()

print(x)

1231120


`print(x)` 의 결과가 0으로 나오는 게 정상적으로 작동할 것이라 생각이 들지만, 실제 계산을 통해 나온 값은 전혀 이상한 숫자가 된다.  
전역 변수 x 에 두 개의 thread가 동시에 접근해서 각자의 작업을 하면서 어느 한 쪽의 작업 결과가 반영이 되지 않기 때문이다.  
이렇게 여러 thread가 공유된 데이터를 변경함으로써 발생하는 문제를 `race condition`이라고도 부른다.

### mutex
Thread-safe한 코드를 만들기 위해서 사용하는 것 중 하나가 mutex (mutual exclusion) 이다. 위에서 본 참사를 막기 위해서, 공유되는 메모리의 데이터를 여러 thread가 동시에 사용할 수 없도록 잠그는 일을 mutex가 맡는다.  

**mutex에 대한 좋은 비유**
> 휴대폰이 없던 시절에는 공중 전화를 주로 이용했었다. 거리의 모든 남자들은 각자의 아내에게 전화를 너무나 걸고 싶어한다.  
어떤 한 남자가 처음으로 공중 전화 부스에 들어가서 그의 사랑하는 아내에게 전화를 걸었다면, 그는 꼭 전화 부스의 문을 꼭 잡고 있어야 한다. 왜냐하면 사랑에 눈이 먼 다른 남자들이 전화를 걸기 위해 시도때도 없이 달려들고 있기 때문이다. 줄 서는 질서 문화 따위는 없다. 심지어 그 문을 놓친다면, 전화 부스에 들이닥친 남자들이 수화기를 뺏어 당신의 아내에게 애정 표현을 할 지도 모른다.  
아내와의 즐거운 통화를 무사히 마쳤다면, 이제 문을 잡고 있던 손을 놓고 부스 밖으로 나가면 된다. 그러면 공중 전화를 쓰기 위해 달려드는 다른 남자들 중 제일 빠른 한 명이 부스에 들어가서 똑같이 문을 꼭 잡고 그의 아내와 통화할 수 있다.

- thread: 각 남자들
- mutex: 공중 전화 부스의 문
- lock: 그 문을 잡고 있는 남자의 손
- resource: 공중 전화

In [9]:
import sys

In [10]:
a = []
b = a

In [11]:
sys.getrefcount(a)

3

- `a`가 처음 만들어 졌을 때의 reference 개수가 하나,
- `b`에 `a`의 reference를 할당했으므로, 그 개수가 하나 늘어나서 두 개,
- `sys.getrefcount`함수에 argument로 `a`가 들어가서, 이 함수 내부에서 `a`의 reference 개수를 하나 늘리므로 세 개 (그리고 이 함수가 끝날 때 다시 reference 개수를 하나 줄일 것이다)  

그리고 이 개수가 0이 되면 CPython이 알아서 메모리를 회수한다고 생각할 수 있다.

In [12]:
import gevent
import random

In [14]:
def task(pid):
    gevent.sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)
    
def synchronous():
    for i in range(1, 10):
        task(i)
        
def asynchronous():
    threads = [gevent.spawn(task, i) for i in range(10)]
    gevent.joinall(threads)
    
print('Synchronous: ')
synchronous()

print('Asynchronous: ')
asynchronous()

Synchronous: 
Task 1 done
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Asynchronous: 
Task 5 done
Task 1 done
Task 2 done
Task 3 done
Task 6 done
Task 7 done
Task 0 done
Task 4 done
Task 8 done
Task 9 done


In [1]:
import gevent.monkey
gevent.monkey.patch_socket()
import gevent
import requests
# import simplejson as json
import json

In [6]:
def fetch(pid):
    response = requests.get('https://api.github.com/events')
    json_result = response.json()
#     json_result = json.loads(result)
    result_id = json_result[0]['id']
    print('Process %s: %s' % (pid, result_id))
    return result_id

def synchronous():
    for i in range(1, 10):
        fetch(i)
        
def asynchronous():
    threads = []
    for i in range(1, 10):
        threads.append(gevent.spawn(fetch, i))
    gevent.joinall(threads)

In [7]:
print('Synchronous:')
synchronous()
print('Asynchronous:')
asynchronous()

Synchronous:
Process 1: 10764816613
Process 2: 10764817224
Process 3: 10764817847
Process 4: 10764818351
Process 5: 10764818973
Process 6: 10764819662
Process 7: 10764820238
Process 8: 10764820926
Process 9: 10764821668
Asynchronous:
Process 1: 10764822209
Process 2: 10764823319
Process 9: 10764823431
Process 6: 10764823431
Process 7: 10764823523
Process 3: 10764823632
Process 4: 10764823632
Process 8: 10764823737
Process 5: 10764823737


In [8]:
import time

In [9]:
def echo(i):
    time.sleep(0.001)
    return i

In [24]:
from multiprocessing import Pool

In [25]:
p = Pool(10)

In [26]:
run1 = [a for a in p.imap_unordered(echo, range(10))]
run2 = [a for a in p.imap_unordered(echo, range(10))]
run3 = [a for a in p.imap_unordered(echo, range(10))]
run4 = [a for a in p.imap_unordered(echo, range(10))]

In [14]:
print(run1 == run2 == run3 == run4)

False


In [15]:
from gevent.pool import Pool

In [16]:
p = Pool(10)
run1 = [a for a in p.imap_unordered(echo, range(10))]
run2 = [a for a in p.imap_unordered(echo, range(10))]
run3 = [a for a in p.imap_unordered(echo, range(10))]
run4 = [a for a in p.imap_unordered(echo, range(10))]

In [17]:
print(run1 == run2 == run3 == run4)

True
